# Tolong gunakan apapun yang anda dapat dari NoteBook ini sebebas-bebasnya, dengan mencantumkan penulis

In [ ]:
# This Python 3 environment comes with many helpful analytics libraries installed  lgebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the "../input/" directory.
# For example, running this (by clicking run or pressing Shift+Enter) will list the files in the input directory

import os
print(os.listdir("../input"))

# Any results you write to the current directory are saved as output.

In [ ]:
import random
import math
from statistics import mean as rerata
import numpy as np

In [ ]:
############################################################################################################

"""
README:

kode decision tree ini diilhami dari program decision tree serta kuliah yang diberikan oleh Josh Gordon melalui youtube
channel google developer dan github dengan username random-forests

link kuliah :
https://www.youtube.com/watch?v=LDRbO9a6XPU&t=323s

link GitHub :
https://github.com/random-forests/tutorials/blob/master/decision_tree.ipynb

dengan perubahan perubahan di antaranya tapi bukan sebatas :

- pembuatan dokumentasi
- mengubah kode sehingga dapat mengakomodasi dataframe pandas
- mengubah kode untuk memenuhi kebutuhan dataset yang dipenuhi oleh fitur kategori
- mengembangkan algoritma sehingga mampu menerima max_depth, dan kolom kelas untuk 
- perubahan lain yang mungkin belum tercantum

algoritma dari decision tree sendiri ialah [1][2][3] :

1. buat root
2. cari kondisi yang paling tinggi info_gainnya menggunakan gini diversity index
3. buat cabang berdasarkan kondisi tersebut
4. jika tercapai max_depth atau tidak terdapat informasi baru yang bisa didapat dari kondisi, hentikan membuat branch
baru

"""

#Decision tree yang digunakan di bawah merupakan decision tree dengan algoritma CART, CART dipilih 
#karena kecepatan yang konsisten dalam melakukan training dan test [1]

def elemen_unik(dataframe, fitur):
    #mengembalikan elemen unik dalam suatu fitur
    return set(dataframe[fitur])

def hitung_kelas(dataframe, kolomkelas = "class"):
    #mengembalikan dictionary fitur class dari dataframe dengan value jumlah
    return dict(dataframe[kolomkelas].value_counts())

#node daun untuk mereturn dictionary dari kelas
class daun:
    def __init__(self, dataframe, kolomkelas):
        self.klasifikasi = hitung_kelas(dataframe,kolomkelas)

#node menyimpan kondisi, dataframe cabang benar, dan dataframe cabang salah
class node_pilihan:
    def __init__(self,
                 kondisi,
                 cabang_benar,
                 cabang_salah):
        self.kondisi = kondisi
        self.cabang_benar = cabang_benar
        self.cabang_salah = cabang_salah

class kondisi:
    
    #inisialisasi class
    def __init__(self, fitur, pembanding):
        self.fitur = fitur
        self.pembanding = pembanding
    
    #menyimpan kondisi
    def banding(self, data):
        return data[self.fitur] == self.pembanding
    
    #digunakan untuk mencetak kondisi
    def __repr__(self):
        condition = "=="
        return "%s %s %s?" % (self.fitur, condition, self.pembanding)

#membagi dataframe berdasarkan kondisi
def bagidataframe(dataframe, kond):
    
    data_benar = dataframe[dataframe[kond.fitur] == kond.pembanding]
    data_salah = dataframe[dataframe[kond.fitur] != kond.pembanding]
    
    return data_benar, data_salah

#ginidx digunakan untuk menemukan gini impurity (i)= 1-zigma(p(kelas)^2) sumber : [2][3]
def giniimpurity(dataframe, kolomkelas):
    kamuskelas = hitung_kelas(dataframe, kolomkelas)
    impurity = 1
    
    for kelas in kamuskelas:
        
        prob_kelas = kamuskelas[kelas] / float(len(dataframe))
        impurity -= prob_kelas**2
        
    return impurity

#mencari information gain (delta i = impurity prior - prob1*impurity1 - prob2*impurity2)sumber: [2][3]
def info_gain(cabang_benar, cabang_salah, current_uncertainty, kolomkelas):
    
    prior_prob = float(len(cabang_benar)) / (len(cabang_benar) + len(cabang_salah))
    info =\
    current_uncertainty - (prior_prob * giniimpurity(cabang_benar, kolomkelas)) -\
    (1 - prior_prob) * giniimpurity(cabang_salah,kolomkelas)

    return info

#metode ini digunakan untuk mencari kondisi terbaik serta 
def cabang_terbaik(dataframe, kolomkelas):
    kondisi_terbaik = None
    gain_terbaik = 0
    ketidakpastian = giniimpurity(dataframe, kolomkelas)
    
    #mengiterasi tiap fitur dalam dataclass
    for fitur in dataframe:
        
        #skip fitur class
        if fitur=="class":
            continue
        
        #mengambil elemen unik dalam suatu fitur    
        kumpulan_tes = elemen_unik(dataframe,fitur)
        
        for tes in kumpulan_tes:
            
            #mengisi kondisi dan membagi dataframe berdasar kondisi
            Kondisi = kondisi(fitur, tes)
            kumpulan_benar,kumpulan_salah = bagidataframe(dataframe, Kondisi)
            
            #jika kondisi tidak membagi dataset continue
            if len(kumpulan_benar) == 0 or len(kumpulan_salah) == 0:
                continue
            
            #mencari information gain untuk kondisi terbaik
            gain = info_gain(kumpulan_benar, kumpulan_salah, ketidakpastian, kolomkelas)
            
            #menyimpan information gain terbaik
            if gain > gain_terbaik : gain_terbaik, kondisi_terbaik = gain, Kondisi
                
    return gain_terbaik, kondisi_terbaik

        
def bangun_pohon(dataframe, max_tinggi, kolomkelas):
    #init gain/prior impurity dan kondisi
    gain, kondisi = cabang_terbaik(dataframe, kolomkelas)
    
    #jika tidak didapat info atau mencapai max tinggi
    
    if gain == 0 or max_tinggi==0:
        return daun(dataframe, kolomkelas)
    
    #mengurangi max tinggi tiap kali bangun pohon dipanggil
    max_tinggi -= 1
    
    #membagi data frame
    cabang_benar, cabang_salah = bagidataframe(dataframe, kondisi)
    
    #perintah untuk  meneruskan pembangunan tree
    cabang_benar = bangun_pohon(cabang_benar, max_tinggi, kolomkelas)
    cabang_salah = bangun_pohon(cabang_salah, max_tinggi, kolomkelas)
    
    #return kondisi dan node benar, salah
    return node_pilihan(kondisi, cabang_benar, cabang_salah)

###############################################################################################################

#digunakan untuk mengklasifikasi data test
def classify(data, node):
     
    #jika node yang dicek adalah daun
    if isinstance(node, daun):
        return node.klasifikasi
    
    #jika data yang di banding dengan kondisi dalam node maka data masukkan klasify ke node yang cabangnya benar
    if node.kondisi.banding(data):
        return classify(data, node.cabang_benar)
    
    #jika if salah
    else:
        return classify(data, node.cabang_salah)
    
##############################################################################################################


#mencetak tree hasil training
def print_tree(node, jeda=""):
    
    #jika  node yang dipilih merupakan daun print hasil prediksi
    if isinstance(node, daun):
        print (jeda + "Prediksi", node.klasifikasi)
        return
    
    print (jeda + str(node.kondisi))

    print (jeda + '--> True:')
    print_tree(node.cabang_benar, jeda + "  ")

    print (jeda + '--> False:')
    print_tree(node.cabang_salah, jeda + "  ")
    
#############################################################################################################

In [ ]:
"""
Berikut dalah algoritma Random Forest (ensemble classifier) dengan bagging pada bagian fitur dataframe
serta sampling berupa N data dengan replacement (Briemann) serta mencoba pohon sehingga bisa sedalam mungkin[4]

Random forest sendiri dipilih karena jumlah fitur dan data dari data yang cukup banyak, sera tipe data yang kategorikal
seluruhnya sangatlah cocok untuk digunakan oleh Random Forest dibandingkan beberapa data lain misal
ADAboost, Fuzzy C-means, dan DBScan

tidak dipilihnya

- Fuzzy C-means : memakan waktu komputasi yang besar, tidak bisa handle missing value [5]

- DBScan        : kurang baik menghadapi data berdimensi tinggi dan densitas yang bervariasi 
                  serta random forest lebih kuat dalam pengerjaan data category [6]
                  
- ADAboost      : Algoritma adaboost yang melakukan boosting terhadap tiap feature serta memberikan optimisasi
                  bahkan terhadap data noise sekalipun dirasa kurang dibutuhkan [7][8][9]

- Ripper        : Random Forest lebih robust terhadap overfit dan jumlah data yang cukup besar 8000++ memberi
                  Random Forest semakin kecil kemungkinan overfit

Masih banyak hipotesis di atas masih membutuhkan pengujian lebih lanjut, alasan utama Random Forest dipilih karena 
memilki reputasi yang sangat baik sebagai klasifier out of the box.

"""

class CARTRF:

    def __init__ (self, dataframe, pohon = 3, tinggi_pohon = 5, kolomkelas = "class"):
        
        kumpulan_fitur = list(dataframe.columns)
        #menyimpan kumpulan fitur namun menghapus kolom target
        kumpulan_fitur.remove(kolomkelas)
        self.trees = []
        
        #training pohon dengan sample with replacement
        for index in range(pohon):
            
            idxfitur = kumpulan_fitur.index(random.choice(kumpulan_fitur))
            
            #menghapus satu fitur secara acak
            train = dataframe.drop(kumpulan_fitur.pop(idxfitur), axis = 1)
            
            #membuat sample with replacement
            sample = train.sample(n = len(train), replace = True)
            
            #menggabung tree yang sudah jadi ke dalam list trees
            self.trees.append(bangun_pohon(sample,tinggi_pohon,kolomkelas))
    
    #klasifikasi
    def klasif(self, data):
        predik = []
        
        #mengiterasi semua pohon di Random Forest trees
        for tree in self.trees:
            #classify data ke dalam tree
            hasil = classify(data ,tree)
            
            #mengambil key dengan value tertinggi
            hasil = max(hasil, key=hasil.get)
            
            #list kunci misal ["e", "e", "p"]
            predik.append(hasil)
            
            #return prediksi yang anggotanya paling sering keluar ["e", "e", "p"] == "e"
        return max(set(predik), key=predik.count)

In [ ]:
def cross_val(dataframe, metode, n = 5, classname = "class"):
    
    #init variabel yang dibutuhkan untuk membagi data menjadi n bagian
    panjang_data = len(dataframe)
    bantu_kelompok = math.floor(panjang_data/n)
    index = 0
    data = []
    
    for i in range(n):
        #jika mencapai i akhir maka data[i] = data ke index sampai data terakhir
        if i == n-1:
            data.append(dataframe[index:(panjang_data)])
            
        #selain itu data ke i adalah data ke index ditambah bantu_kelompok
        else:
            data.append(dataframe[index : (index+bantu_kelompok)])
        
        #tambah index    
        index += bantu_kelompok
    
    #buat list untuk menyimpan akurasi tiap iterasi untuk diprint
    akurasi = []
    
    #test dan train dilakukan n kali
    for i in range(n):
        
        #buat train dan tes
        train = None
        tes = data[i]
        
        #concat data train
        for iterasi in range(n):
            #jika iterasi sama dengan nilai iterasi sebelumnya, maka tidak perlu di concat karena menjadi tes
            if iterasi != i:  
                train = pd.concat([train, data[i]], join = "inner")
        
        #membuat classifier dan variabel prediksi
        classifier = metode(train)
        prediksi = []
        kelas_asli = tes[classname]
        
        #prediksi
        for ID, coba in tes.iterrows():
            prediksi.append(classifier.klasif(coba))
            
        jumlah_benar = (np.array(prediksi) == np.array(tes[classname])).sum()
        akurasi.append((jumlah_benar/len(tes))*100)
        
    return(akurasi)

In [ ]:
dataframe = pd.read_csv("../input/mushrooms.csv")

In [ ]:
dataframe.describe(include="O")

In [ ]:
dataframe.head()

In [ ]:
(dataframe=="?").sum()

In [ ]:
#drop Stalk root karena missing value yang besar, dan veil-type karena hanya ada 1 unique value

print(2480/len(dataframe))

dataframe  = dataframe.drop(["stalk-root", "veil-type"], axis = 1)

In [ ]:
#perbedaan kelas tidak terlalu besar

print(dataframe.groupby("class").size())

In [ ]:
#menggabungkan fitur yang memliki awalan sama untuk mereduksi fitur

cap = dataframe["cap-shape"]+dataframe["cap-surface"]+dataframe["cap-color"]
gill = dataframe["gill-attachment"]+dataframe["gill-spacing"]+dataframe["gill-size"]+dataframe["gill-color"]
stalk = dataframe["stalk-shape"]+dataframe["stalk-surface-above-ring"]+dataframe["stalk-surface-below-ring"]+dataframe["stalk-color-above-ring"]+dataframe["stalk-color-below-ring"]
ring = dataframe["ring-number"]+dataframe["ring-type"]


#drop fitur yang telah di gabung
for fitur in dataframe:
    if "gill" in fitur or "stalk" in fitur or "ring" in fitur or "cap" in fitur:
        dataframe = dataframe.drop(fitur, axis =1)
        
#memasukkan fitur yang sudah digabung ke dalam dataframe

dataframe["gill"] = gill
dataframe["cap"] = cap
dataframe["stalk"] = stalk
dataframe["ring"] = ring

Windows 10, RAM 4 GB, intel core i5 6400u

Evaluasi dengan Pre-process dasar = drop veil type dan max depth = 5 pohon = 3:

1. tanpa pre process tanpa sampling replacement n = 5 [(27.6 second, 99.97%), (26.2 second, 99.97%), (30.7 second, 99.97%)]
2. pre process tanpa sampling replacement n = 5 [(34.5 second, 99.93%), (34.3 second, 100%), (32.1 second, 100%)]
3. tanpa pre process sampling replacement n = 5 [(27.5 second, 99.97%), (28.1 second, 99.96%), (29 second, 99.97%)]
4. pre process sampling replacement n = 5 [(36.1 second, ~100%), (31.8 second, ~100%), (35.7 second, ~100%)]

"""
Dalam melakukan pre-process di atas adalah menggunakan metode seleksi fitur manual dengan kejelasan:
1. Veil-Type hanya memiliki 1 unique value, sehingga tidak bisa digunakan sebagai fitur seleksi
2. Penggabungan beberapa fitur sehingga terbentuk kumpulan pola bentuk tiap bagian jamur
[10][11][12][13]
"""

In [ ]:
%%time

rerata(cross_val(dataframe, CARTRF))

Sumber rujukan :

1. Kumar, Sunil, 2016.  "A Survey on Decision Tree Algorithms of Classification on Data Mining"
2. Leszek Rutkowski, Maciej Jaworski, Lena Pietruczuk, Piotr Duda, 2018. "The CART Decision Tree for Mining Data Streams"
3. Breiman, L., Friedman, J.H., Olshen, R., and Stone, C.J., 1984. Classification and Regression, TreeWadsworth & Brooks/Cole Advanced Books & Software, Pacific California. 
4. Eesha Goel, Er. Abhilasha, 2017. "Random Forest: A Review"
5. R.Suganya, R.Shanthi, 2012. "Fuzzy C- Means Algorithm-  A Review"
6. K. Nafees Ahmed, T. Abdul Razak, 2016. "An Overview of Various Improvements of DBSCAN Algorithm in Clustering Spatial Databases"
7. Abraham J.Wyner, Matthew Olson, Justin Bleich, 2017. "Explaining the Success of AdaBoost and Random Forests as Interpolating Classifiers"
8. Artur Ferreira, M´ario Figueiredo, ----."Boosting Algorithms: A Review of Methods, Theory, and Applications"
9. Robert E. Schapire, ----. "Explaining ADABoost"
10. Wesam S. Bhaya, 2017. "Review of Data Preprocessing Techniques in Data Mining"
11. Ms. Shweta Srivastava, Ms. Nikita Joshi, Ms. Madhvi Gaur ,2013 . "A Review Paper on Feature Selection Methodologies and Their Applications"
12. Isabelle Guyon, Andre Elisseeff , 2013. "An Introduction to Variable and Feature Selection"
13. Jianyu Miao, Lingfeng Niu, 2016. "A Survey on Feature Selection"

With love and passion :

Contact Info:

- LinkedIn : https://www.linkedin.com/in/fais-alqorni-9a7642172/
- Kaggle   : https://www.kaggle.com/paperbagz/
- e-Mail   : faisqorni12@gmail.com

Fais Alqorni 
Malang, 24/04/2019